In [3]:
pip install setlocale

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement setlocale (from versions: none)
ERROR: No matching distribution found for setlocale


In [8]:
"""
                             Script:
                             Les données qui vont sont mis à disposition concernent 3 grandes villes au USA (New York, Chicago et Washington).
                             Les données concernent:
                             les 6 premiers mois de l'année 2017 pour New York,
                             puis Juillet, Aout et Septembre 2017 pour Chicago
                             et enfin, Octobre, Novembre et Décembre 2017 pour Washington.

"""

# Import de modules
import pandas as pd
import matplotlib.pyplot as plt
# Pour la boite de dialogue
import tkinter as tk
from tkinter import simpledialog
# Pour les jours de la semaine en français
from locale import setlocale, LC_TIME
setlocale(LC_TIME, 'fr_FR.UTF-8')
import ipywidgets as widgets

# Import des fichiers de Chicago, New York et Washington
data_chicago = pd.read_csv("cleaned_chicago_city.csv")
data_newyork = pd.read_csv("cleaned_new_york_city.csv")
data_washington = pd.read_csv("cleaned_washington_city.csv")

# Je convertis les datetime pour après récuperer les mois et jours
data_chicago['Start Time'] = pd.to_datetime(data_chicago['Start Time'])
data_chicago['End Time'] = pd.to_datetime(data_chicago['End Time'])

data_newyork['Start Time'] = pd.to_datetime(data_newyork['Start Time'])
data_newyork['End Time'] = pd.to_datetime(data_newyork['End Time'])

data_washington['Start Time'] = pd.to_datetime(data_washington['Start Time'])
data_washington['End Time'] = pd.to_datetime(data_washington['End Time'])

# Création de dictionnaire pour récuperer le int de mois

dict_mois = {
    1: "Janvier", 
    2: "Février",
    3: "Mars",
    4: "Avril",
    5: "Mai",
    6: "Juin",
    7: "Juillet",
    8: "Août",
    9: "Septembre",
    10: "Octobre",
    11: "Novembre",
    12: "Décembre"
}

# Choix des 3 villes
villes = ['Chicago', 'New York', 'Washington']
boutons_villes = widgets.RadioButtons(options=villes, description='Villes :')

# Pour les mois
boutons_mois = widgets.RadioButtons(options=dict_mois.values(), description='Mois :')

# Pour afficher les résultats à droite
resultats_html = widgets.HTML(value="<b><h1>********** TABLEAU DE BORD **********</h1></b>")

# Fonction pour filtrer la table avec le mois souhaité avec le Start Time
def filtrer_data(ville_data, mois):
    return ville_data[ville_data['Start Time'].dt.month == list(dict_mois.keys())[list(dict_mois.values()).index(mois)]]

# Fonctions pour les statistiques
def statistiques(ville, mois):
    if ville == 'Chicago':
        resultat = filtrer_data(data_chicago, mois)
    elif ville == 'New York':
        resultat = filtrer_data(data_newyork, mois)
    elif ville == 'Washington':
        resultat = filtrer_data(data_washington, mois)
    else: 
        # retourne un dataframe vide
        resultat = pd.DataFrame()
    
    if not resultat.empty:
        # Dans le cas où on rècupere une table non vide(mois renseigné)

        # 1. Le jour de la semaine avec le plus d'activité
        # je crée une colonne jour pour récuperer les jours de la semaine
        resultat.loc[:, 'jour'] = resultat['Start Time'].dt.strftime('%A')
        nb1 = resultat.groupby('jour').size().rename("nb_jour").sort_values(ascending=False).reset_index()
        meilleur_jour = nb1.iloc[0]['jour']   

        # 2. L'heure de démarrage la plus courante.
        # je crée une colonne heure pour récuperer les heures
        resultat.loc[:, 'heure'] = resultat['Start Time'].dt.hour.apply(lambda x: str(x).zfill(2))
        nb2 = resultat.groupby('heure').size().rename("heure_debut").sort_values(ascending=False).reset_index()
        debut = nb2.iloc[0]['heure']

        # 3. La durée de voyage moyen sur la période (mois). en minute
        duree = resultat['Trip Duration'].mean() / 60

        # 4. Le total pour chaque catégorie de User

        nb3 = resultat.groupby('Gender').size().rename("Genre").sort_values(ascending=False).reset_index()
        femme = nb3.iloc[0]['Genre']
        homme = nb3.iloc[1]['Genre']
        nonrenseigne = nb3.iloc[2]['Genre']
        

    else:
        meilleur_jour = debut = duree = femme = homme = nonrenseigne = 0
    
    return meilleur_jour, debut, duree, femme, homme, nonrenseigne

def maj_resultats(ville, mois):
    meilleur_jour, debut, duree, femme, homme, nonrenseigne = statistiques(ville, mois)
    resultats_html.value = f"<b><h1>********** TABLEAU DE BORD **********</h1></b><br>"\
                           f"Ville sélectionnée : {ville}, Mois sélectionné : {mois}<br>" \
                           f"<b>Jour avec le plus d'activité :</b> {meilleur_jour}<br>" \
                           f"<b>Heure de démarrage la plus courante :</b> {debut}<br>" \
                           f"<b>Durée moyenne des trajets (en minutes) :</b> {duree:.2f}<br>"\
                           f"<b>Le total des femmes est :</b> {femme}<br>"\
                           f"<b>Le total des hommes est :</b> {homme}<br>"\
                           f"<b>Le total du genre non renseigné est :</b> {nonrenseigne}"

bouton_valider = widgets.Button(description="Valider")

def valider_clic(b):
    resultats_html.value = ""  # Réinitialisation du contenu à une chaîne vide
    maj_resultats(boutons_villes.value, boutons_mois.value)

    # maj_resultats(boutons_villes.value, boutons_mois.value)

bouton_valider.on_click(valider_clic)

espace1 = widgets.HTML(value="<br>" * 8)
espace2 = widgets.HTML(value="<br>" * 9)

colonne1 = widgets.VBox([boutons_villes])
colonne2 = widgets.VBox([boutons_mois])
espace3 = widgets.VBox([])
colonne4 = widgets.VBox([resultats_html])

display(widgets.HBox([colonne1, colonne2, espace3, colonne4]))
display(widgets.HBox([espace1, bouton_valider, espace2]))

C:\Users\MDIENG\AppData\Local\Temp\112\ipykernel_7876\2069641830.py:25: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  data_washington = pd.read_csv("cleaned_washington_city.csv")
